In [ ]:
# import necessary packages
import requests
import json
import pandas as pd

In [ ]:
# client info
client_id = '6b23af5a68764b19b9a90fc8b45c1903'
client_secret = 'e9351845da054ea7a05df93dc52495dc'

In [ ]:
# auth url
auth_url = f'https://accounts.spotify.com/authorize?client_id={client_id}&response_type=code&redirect_uri=http://localhost'
auth_url

In [ ]:
# auth code from url
code = 'AQAsEXJGphPs4c9w61VEzoYiNa9_k5CZxL-p70UG1CAEWFjSCn9d3kd6fsTzxcPmNN7wOV_UrmdXaKm1paRwIgtEsP0wlkGjv_eo1hcCUeP9xSFBDkbx3qKUwc4w95Ozs_f4BzoiWASrAJesaBouJgVGyN4'

In [ ]:
# request token
token_endpoint_url = 'https://accounts.spotify.com/api/token'

request_body = {
    'grant_type':'authorization_code',
    'code':code,
    'redirect_uri':'http://localhost',
    'client_id':client_id,
    'client_secret':client_secret
}

token_api_request = requests.post(token_endpoint_url, data = request_body)

In [ ]:
json.loads(token_api_request.text)

In [ ]:
token_api_response = json.loads(token_api_request.text)

In [ ]:
# save token to variable
oauth_token = token_api_response['access_token']
oauth_token

In [ ]:
# pass token to header
headers = {'Authorization':f'Bearer {oauth_token}'}
headers

In [ ]:

query = "genre:'house'"



In [ ]:
# set params and url

url = 'https://api.spotify.com/v1/search'
params = {
        'q': query,
        'type': 'artist',
        'limit': 50
}


In [ ]:
# save response
response = requests.get(url, headers=headers, params=params)


In [ ]:
# view response
json.loads(response.text)

In [ ]:
artist_data = json.loads(response.text)

In [ ]:
# save artist info to artist_details list

artist_details = []

for artist in artist_data['artists']['items']:
    id = artist['id']
    name = artist['name']
    popularity = artist['popularity']
    followers = artist['followers']['total']
    genres = artist['genres']
    artist_details.append({'id': id,'name': name, 'popularity': popularity, 'followers': followers, 'genres': genres})

print(artist_details)

In [ ]:
# send more requests to get more than 50 artists
total_artists = artist_data['artists']['total']
num_requests = 0
if total_artists > 50:
    num_requests = (total_artists - 50) // 50
    if (total_artists - 50) % 50 != 0:
        num_requests += 1


In [ ]:
# send more requests to get more than 50 artists
for i in range(num_requests):
    params['offset'] = (i + 1) * 50
    response = requests.get(url, params=params, headers=headers)
    additional_data = response.json()
    for artist in additional_data['artists']['items']:
        id = artist['id']
        name = artist['name']
        popularity = artist['popularity']
        followers = artist['followers']['total']
        genres = artist['genres']
        artist_details.append({'id': id,'name': name, 'popularity': popularity, 'followers': followers, 'genres': genres})

print(artist_details)


[{'id': '7kNqXtgeIwFtelmRjWv205', 'name': 'John Summit', 'popularity': 68, 'followers': 274117, 'genres': ['edm', 'house', 'pop dance', 'uk dance']}, {'id': '205i7E8fNVfojowcQSfK9m', 'name': 'Dom Dolla', 'popularity': 66, 'followers': 379369, 'genres': ['australian house', 'deep groove house', 'house']}, {'id': '4oLeXFyACqeem2VImYeBFe', 'name': 'Fred again..', 'popularity': 76, 'followers': 1416255, 'genres': ['edm', 'house', 'stutter house']}, {'id': '5Igpc9iLZ3YGtKeYfSrrOE', 'name': 'Chris Lake', 'popularity': 65, 'followers': 323112, 'genres': ['bass house', 'edm', 'electro house', 'house', 'melbourne bounce', 'pop dance', 'progressive house', 'tech house']}, {'id': '1VJ0briNOlXRtJUAzoUJdt', 'name': 'FISHER', 'popularity': 72, 'followers': 917855, 'genres': ['australian house', 'house']}, {'id': '2mLA48B366zkELXYx7hcDN', 'name': 'Peggy Gou', 'popularity': 66, 'followers': 906770, 'genres': ['electra', 'float house']}, {'id': '6nS5roXSAGhTGr34W6n7Et', 'name': 'Disclosure', 'popularit

In [ ]:
# save artist_details to a data frame and explode the genres
artist_details_df = pd.DataFrame(artist_details)
artist_details_df = artist_details_df.explode('genres')

print(artist_details_df)


                         id         name  popularity  followers  \
0    7kNqXtgeIwFtelmRjWv205  John Summit          68     274117   
0    7kNqXtgeIwFtelmRjWv205  John Summit          68     274117   
0    7kNqXtgeIwFtelmRjWv205  John Summit          68     274117   
0    7kNqXtgeIwFtelmRjWv205  John Summit          68     274117   
1    205i7E8fNVfojowcQSfK9m    Dom Dolla          66     379369   
..                      ...          ...         ...        ...   
996  5srbzu0tyZ1Wb1HgDOqVFj       Modera          38       4000   
997  23iMeGaHEpDoHAojo4JeVy         YVMV          44        348   
998  3mYipWrjv0ksO0s9R5Zdfy    Paris Blu          44       4716   
999  6zZWFyAqVLEWelkVuQRUCz   Playahitty          46      12128   
999  6zZWFyAqVLEWelkVuQRUCz   Playahitty          46      12128   

               genres  
0                 edm  
0               house  
0           pop dance  
0            uk dance  
1    australian house  
..                ...  
996     melodic house  
997

In [ ]:
artist_details_df.to_csv('artist_details.csv', index=False)